# 신경망(Neural Networks)

torch.nn 패키지를 사용
- 학습가능한 매개변수(or 가중치)를 갖는 신경망 정의
- 데이터셋(dataset) 입력을 반복
- 입력을 신경망에서 처리
- 손실(loss)를 계산
- 변화도(gradient)를 backpropagation
- 신경망의 가중치 갱신
    - weight = weight - learning rate * gradient

## torch.nn

##### torch.nn.Module
- 모든 neural network module의 base class
- nn.Moudle을 상속받아 서브클래스를 만들고 forward를 정의
- forward 안에서는 입력 Variable을 받아 다른 모듈과 다른 autograd 연산을 이용하여 출력 Variable을 만드는 역할

##### nn.Conv2d
- nn.Conv2d(in channels, out channels, kernel size, stride=1, padding=0, dilation=1, groups=1, bias=True)
- Parameters:
    - in_channels(int) : input의 channel 수
    - out channels(int) : convolution에의해 생성된 channel의 수
    - kernel_size(int or tuple) : convolution kernel의 크기
    - stride(int or tuple) : convolution의 stride
    - padding(int or tuple) : zero padding의 크기
    - dilation(int or tuple) : kernel element사이의 공간
    - group : group만큼 input channel을 나누고 각각의 그룹에 대해 독립적으로 convolution 수행후 그 결과를 concatenation
    - bias : bias의 유무
- Shape:
    - Input : (N, C_in, H_in, W_in)
    - Output : (N, C_out, H_out, W_out)
    - N: batch size
    - H_out = ceil((H_in + 2 * padding - dilation * kernel_size - 1)/stride) + 1
- Variable:
    - weight : learnable weight shape(out_channel, in_channel, kernel_size[0], kernel_size[1])
    - bias : learnable bias (out_channels)

##### nn.Linear
- nn.Linear(in_features, out_features, bias=True)
- input에 y=xA^T+b의 linear transform 적용
- Parameters:
    - in_features : input sample의 size
    - out_features : output sample의 size
    - bias : b의 유무 default=True
- Shape:
    - Input(N, *, in_features) : * any number of additional demension
    - Output(N, *, out_features)
- Variables:
    - weight : learnable weights shape(out_features * in_features)
    - bias : learnable bias shape(out_features)


In [4]:
#neural net 정의
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] #batch size를 제외한 모든 size
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


- nn.Module을 상속받아서 만든 클래스를 함수형으로 call하면 forward가 호출됨
- forward의 입력과 출력은 autograd.Variable

In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [59]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

tensor([[ 0.0087,  0.0399, -0.0940, -0.1718, -0.1074, -0.0035, -0.1137,  0.1068,
         -0.1178, -0.1762]], grad_fn=<ThAddmmBackward>)


In [36]:
net.zero_grad()
#backward를 수행하면 이전에 수행했던 forward결과를 reset
out.backward(torch.randn(1,10))

## Loss Function
loss function : (output, label)을 입력으로 받아 output과 label(정답)이 얼마나 가까운지 추정하는 값을 계산하는 함수
nn패키지에는 여러 loss function이 존재 ex) nn.MSEloss

In [64]:
input = Variable(torch.randn(1, 1, 32, 32))
output = net(input)
target = Variable(torch.arange(1, 11).float())  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
print(type(target))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

<class 'torch.Tensor'>
tensor(39.3349, grad_fn=<MseLossBackward>)


## Backprop
gradient function을 계산해서 각 Variable마다 gradient update
- zero_grad() : gradient buffer를 0으로 초기화 - 기존의 gradient가 존재하면 초기화
- backward() : gradient function 계산 후 gradient update

In [68]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0101,  0.1119,  0.0005, -0.0174,  0.0135,  0.0606])


## Weight update
- 계산한 gradient를 바탕으로 각 parameter 가중치 update
- 가장 간단한 update rule은 SGD(Stochastic Gradient Descent): weight = weight - learning rate * gradient


In [69]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

신경망을 구성할 때 다양한 optimization 방법들은 이미 torch.optim에 구현되어 있음
(Adam, SGD, RMSProp 등)

In [73]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.01)

#수동으로 gradient buffer를 비워줘야함 - 비워주지 않으면 값이 누적됨
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()